# DLS project by Alexandr Korchemnyi

In [17]:
# !git clone https://github.com/ultralytics/yolov5

In [2]:
%cd yolov5
# %pip install -qr requirements.txt

/home/yessense/projects/dls project/yolov5


In [19]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [20]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.8.1+cu102 (NVIDIA GeForce RTX 2060)


In [21]:
dataset_directory = "/home/yessense/projects/dls project/dataset"

In [ ]:
train_len = len(os.listdir(os.path.join(dataset_directory, "train/images")))
test_len = len(os.listdir(os.path.join(dataset_directory, "test/images")))
valid_len = len(os.listdir(os.path.join(dataset_directory, "valid/images")))

print(f'Lenght of train dataset: {train_len}')
print(f'Lenght of validation dataset: {valid_len}')
print(f'Lenght of test dataset: {test_len}')
print(f'Total lenght with reflect augmentation: {sum([train_len, valid_len, test_len])}')


# Данные
Набор данных снят на полигоне для дронов в МИРЭА. Размечен с помощью roboflow.

![](imgs/train_batch0.jpg)

# Тренировка модели
Изначальная модель - yolov5n
Взята, как самая маленькая, чтобы запускать на

In [ ]:
!python train.py --img 320 --batch 16 --epochs 150 --data "/home/yessense/projects/dls project/dataset/data.yaml" --weights yolov5n.pt --cache --hyp "/home/yessense/projects/dls project/dataset/hyp.yaml"

![](imgs/F1_curve.png)
![](imgs/P_curve.png)
![](imgs/PR_curve.png)
![](imgs/R_curve.png)

In [ ]:
!python export.py --img 320 --batch 1 --weights "/home/yessense/projects/dls project/yolov5/runs/train/exp5/weights/best.pt" --include onnx

# Результаты

